In [1]:
import numpy as np
from skimage.measure import label, regionprops
from skimage.util import invert
from scipy.sparse import csr_matrix,lil_matrix,coo_matrix
from scipy.linalg import eigh, inv, logm, norm
from  scipy import ndimage,sparse
import cv2
import os
import sys
import csv
import glob

import h5py
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
'''Using histomicstk'''
import histomicstk as htk

import numpy as np
import scipy as sp

import skimage.io
import skimage.measure
import skimage.color

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline


In [15]:
import pandas as pd

#csv_file = '../pkl/csv/id_13.fov_centroids_embedding_morphology.covd.pkl.csv'
csv_file = '../pkl/id_13.fov_centroids_embedding_morphology.covd.pkl.csv'
df = pd.read_csv(csv_file)

#total_mask_label = np.zeros((512+512*df['fov_row'].max(),512+512*df['fov_col'].max()))

rc_df = df.groupby(['fov_row','fov_col']).size().reset_index().rename(columns={0:'count'}) # the df of row-col fov pairs

for row, col in list(zip(rc_df.fov_row, rc_df.fov_col))[:2]:
    print(row,col)
    df_fov = df.loc[(df['fov_row'] == row) & (df['fov_col'] == col)]

    # load the h5 file
    h5_file = '../h5/id_13/iMS342_20190715_001._r'+str(row)+'_c'+str(col)+'.h5'   #this file contains the segmented nuclei                                                                                                                    
    fov = h5py.File(h5_file, 'r') # load the current fov segmentation                                                                                                                   
    mask = fov['/exported_watershed_masks'][:]
    mask_reduced = np.squeeze(mask, axis=2) #to get rid of the third dimension                                                                                                          
    mask_label, numb_of_nuclei = label(mask_reduced,return_num=True) # to define the mask_label array
    
    # load dapi file
    basename = os.path.splitext(h5_file)[0]
    dapi_file = basename+'.tif' # the dapi file has to be located in the same directory as the h5 file                                                                                  
    dapi_fov= cv2.imread(dapi_file,cv2.IMREAD_GRAYSCALE) #the dapi tif file of the current FOV                                                                                          

    # define new_mask_label
    new_mask_label = np.zeros((512,512))
    for index, nuclei in df_fov.iterrows(): # for each nucleus in csv in fov
        # read the necessary information from each row
        cx = nuclei['cx']
        cy = nuclei['cy']
        cluster = int(nuclei['cluster'])
        for region in regionprops(mask_label,intensity_image=dapi_fov): # for each nucleus in fov
            #if a region is too small or too large do not consider it
            if not ((np.count_nonzero(region.intensity_image) <= 10) or (np.count_nonzero(region.intensity_image) > 2500)) :
                x = 512*int(col)+region.centroid[0] # shift by FOV location                                                                                                                 
                y = 512*int(row)+region.centroid[1] # shift by FOV location                                                                                                                 
                if x == cx and y == cy:
                    # find location in mask_label where label is equal to region.label
                    new_mask_label[mask_label == region.label] = cluster + 1 # sum 1 to make unclustered nuclei equal to bg_label 0

 #   total_mask_label[512*row:512*(row+1),512*col:512*(col+1)] = new_mask_label


    # show pre-clustering figure
    #plt.subplot(1, 2, 1)
#    plt.figure(figsize=(10, 10))
    plt.imsave(basename+'.pre-clustering.png',
               invert(skimage.color.label2rgb(mask_label, dapi_fov, bg_label=0)), 
               origin='lower')
#    plt.plot(invert(skimage.color.label2rgb(mask_label, dapi_fov, bg_label=0)), origin='lower')

    # show post-clustering figure
    #plt.subplot(1, 2, 2)
#    plt.figure(figsize=(10, 10))
    plt.imsave(basename+'.post-clustering.png',
               invert(skimage.color.label2rgb(new_mask_label, dapi_fov, bg_label=0)), origin='lower')
    
#    plt.savefig(basename+'.PrePost-clustering.png')
                    

0 5
0 6


There are cell missing...why?

In [12]:
# label all connected components in the fov, 0 is background                                                                                                                        
mask_label, numb_of_nuclei = label(mask_reduced,return_num=True)

# Display results
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.imshow(skimage.color.label2rgb(mask_label, dapi_fov, bg_label=0), origin='lower')
plt.title('Nuclei segmentation mask overlay', fontsize=12)

plt.subplot(1, 2, 2)
plt.imshow( dapi_fov )
plt.xlim([0, dapi_fov.shape[1]])
plt.ylim([0, dapi_fov.shape[0]])
plt.title('Nuclei bounding boxes', fontsize=12)
for region in regionprops(mask_label,intensity_image=dapi_fov):
    counter+=1
    #if a region is too small or too large do not consider it
    if ((np.count_nonzero(region.intensity_image) <= 10) or (np.count_nonzero(region.intensity_image) > 2500)) :
        print('The number of pixels is '+str(np.count_nonzero(region.intensity_image))+' in region='+str(counter))
    else:
        c = [region.centroid[1], region.centroid[0], 0]
        width = region.bbox[3] - region.bbox[1] + 1
        height = region.bbox[2] - region.bbox[0] + 1

        cur_bbox = {
            "type":        "rectangle",
            "center":      c,
            "width":       width,
            "height":      height,
        }

        plt.plot(c[0], c[1], 'g+')
        mrect = mpatches.Rectangle([c[0] - 0.5 * width, c[1] - 0.5 * height] ,
                               width, height, fill=False, ec='g', linewidth=2)
        plt.gca().add_patch(mrect)